#### Load necessary libraries

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

#### Filepath for data and some constant

In [281]:
#filepath of file with labels and training set
name = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/meta/meta.txt'
path_training = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/audio/'
#put here filepath for new set
path_test = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/test/' 
n_mfcc = 50
number_fun = 2

#### Reading data

In [282]:
#obtain features and scaling for new set
features_test = np.array([]).reshape(0, n_mfcc*number_fun)
name_test_file = np.array([])
for file in os.listdir(path_test):
    x, sr = librosa.load(os.path.join(path_test,file))
    mfcc = librosa.feature.mfcc(x, sr, n_mfcc = n_mfcc)
    mfcc_mean = np.mean(mfcc.T,axis=0)
    mfcc_median = np.median(mfcc.T,axis=0)
    features_test = np.vstack((features_test, np.concatenate((mfcc_mean, mfcc_median))))
    name_test_file = np.append(name_test_file, file)
test = StandardScaler().fit_transform(features_test)

In [284]:
#transform and obtain features from training audio file 
features = np.array([]).reshape(0, n_mfcc*number_fun)
name_file = np.array([])
for file in os.listdir(path_training):
    x, sr = librosa.load(os.path.join(path_training,file))
    mfcc = librosa.feature.mfcc(x, sr, n_mfcc = n_mfcc)
    mfcc_mean = np.mean(mfcc.T,axis=0)
    mfcc_median = np.median(mfcc.T,axis=0)
    features = np.vstack((features, np.concatenate((mfcc_mean, mfcc_median))))
    name_file = np.append(name_file, file)

In [285]:
#load label of classes
label = pd.read_csv(name, sep = '\t', header=None)
label.columns = ['name', 'place', 'start', 'end', 'class']
features_data = pd.DataFrame(np.concatenate((name_file.reshape(name_file.shape[0],1), features), axis=1))
features_data.columns = np.concatenate((['name'], ['feature_'+str(i) for i in range(1,n_mfcc*number_fun+1)]))
train = pd.merge(label, features_data, on='name')

#### Classifcation

In [286]:
#scaling training set
X = train.drop(['class','name', 'place', 'start', 'end'], axis = 1)
X = StandardScaler().fit_transform(X)

In [287]:
# Binarize the output
y = label_binarize(train['class'], classes=train['class'].unique())

In [288]:
#split set on train and test
train_X, test_X, train_y, test_y = train_test_split(X,y,train_size=0.5, test_size = 0.5,random_state=40)

In [289]:
num_labels = y.shape[1]

In [290]:
# build model
model = Sequential()

model.add(Dense(32, input_shape=(n_mfcc*number_fun,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [299]:
model.fit(train_X, train_y, batch_size=50, epochs=100, validation_data=(test_X, test_y))
scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Train on 5653 samples, validate on 5654 samples
Epoch 1/100
5653/5653 [==============================] - 0s 50us/step - loss: 0.4361 - acc: 0.8696 - val_loss: 0.1867 - val_acc: 0.9484
Epoch 2/100
5653/5653 [==============================] - 0s 46us/step - loss: 0.4335 - acc: 0.8611 - val_loss: 0.1791 - val_acc: 0.9508
Epoch 3/100
5653/5653 [==============================] - 0s 47us/step - loss: 0.4473 - acc: 0.8590 - val_loss: 0.1795 - val_acc: 0.9507
Epoch 4/100
5653/5653 [==============================] - 0s 47us/step - loss: 0.4300 - acc: 0.8652 - val_loss: 0.1737 - val_acc: 0.9545
Epoch 5/100
5653/5653 [==============================] - 0s 48us/step - loss: 0.4428 - acc: 0.8608 - val_loss: 0.1680 - val_acc: 0.9538
Epoch 6/100
5653/5653 [==============================] - 0s 48us/step - loss: 0.4532 - acc: 0.8512 - val_loss: 0.1766 - val_acc: 0.9538
Epoch 7/100
5653/5653 [==============================] - 0s 48us/step - loss: 0.4551 - acc: 0.8578 - val_loss: 0.1774 - val_acc: 0.9545


5653/5653 [==============================] - 0s 46us/step - loss: 0.3706 - acc: 0.8843 - val_loss: 0.1300 - val_acc: 0.9653
Epoch 61/100
5653/5653 [==============================] - 0s 44us/step - loss: 0.3616 - acc: 0.8845 - val_loss: 0.1409 - val_acc: 0.9622
Epoch 62/100
5653/5653 [==============================] - 0s 45us/step - loss: 0.3783 - acc: 0.8794 - val_loss: 0.1374 - val_acc: 0.9614
Epoch 63/100
5653/5653 [==============================] - 0s 44us/step - loss: 0.3805 - acc: 0.8794 - val_loss: 0.1373 - val_acc: 0.9629
Epoch 64/100
5653/5653 [==============================] - 0s 44us/step - loss: 0.3625 - acc: 0.8799 - val_loss: 0.1331 - val_acc: 0.9629
Epoch 65/100
5653/5653 [==============================] - 0s 45us/step - loss: 0.3725 - acc: 0.8817 - val_loss: 0.1260 - val_acc: 0.9655
Epoch 66/100
5653/5653 [==============================] - 0s 45us/step - loss: 0.3575 - acc: 0.8838 - val_loss: 0.1262 - val_acc: 0.9660
Epoch 67/100
5653/5653 [==============================

#### Prediction and results

In [300]:
test_class = model.predict_classes(test)

values = train['class'].unique()
keys = range(num_labels)
dictionary = dict(zip(keys, values))
test_class = [dictionary.get(i) for i in test_class]

In [301]:
test_prob = model.predict(test)

In [302]:
result = pd.DataFrame()

In [303]:
result['file_name'] = name_test_file
result['predict'] = np.max(test_prob, axis = 1)
result['class'] = test_class

In [304]:
result

,file_name,predict,class
0,background_0001.wav,0.908963,keyboard
1,background_0004.wav,0.754397,background
2,background_0009.wav,0.946057,keyboard
3,background_0012.wav,0.682531,keyboard
4,background_0016.wav,0.704889,background
5,background_0018.wav,0.987110,background
6,background_0022.wav,0.949173,keyboard
7,background_0028.wav,0.947109,keyboard
8,background_0031.wav,0.961212,keyboard
9,background_0036.wav,0.917958,keyboard


In [306]:
result.to_csv('C:/Users/Python/Documents/STC_test/result.txt', sep = '\t', header =False, index = False)